# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [74]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
#from pygeocoder import Geocoder
# Import API key
from key import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_df = pd.read_csv(r"C:\Users\yenia\Desktop\JHU_Bootcamp\Homework\Whats-The-Weather-Like\WeatherPy\Data\cities.csv") 
print(len(weather_df))
weather_df.head()

572


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.20,-175.20,78.80,88,40,9.17,TO,1607284012
1,Kudahuvadhoo,2.67,72.89,82.51,78,100,14.29,MV,1607284012
2,Malanje,-9.54,16.34,62.65,95,80,1.57,AO,1607284012
3,Punta Arenas,-53.15,-70.92,68.00,45,75,26.40,CL,1607284012
4,Christchurch,-43.53,172.63,69.80,56,97,9.17,NZ,1607283830


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

In [56]:
#Drop NaN values
clean_weather_df = weather_df.dropna()
len(clean_weather_df)
#Store "Lat" and "Lng" into coordinates
coordinates = clean_weather_df[["Lat", "Lng"]].astype(float)
#store "Humidity"
humidity = clean_weather_df["Humidity"].astype(float)


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
under_80 = clean_weather_df.loc[clean_weather["Max Temp"] > 80,:]
ideal_temp = under_80.loc[under_80["Max Temp"] >70, :]
ideal_winds = ideal_temp.loc[ideal_temp["Wind Speed"] <10,:]
ideal_weather_df = ideal_winds.loc[ideal_winds["Cloudiness"] == 0,:]
print(len(ideal_weather_df))
ideal_weather_df

7


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,Talca,-35.43,-71.67,89.60,24,0,4.70,CL,1607284016
297,Illapel,-31.63,-71.17,81.30,29,0,9.33,CL,1607284030
367,Kaffrine,14.11,-15.55,83.14,23,0,7.25,SN,1607284034
394,Terra Nova,-12.37,-40.67,83.88,44,0,4.47,BR,1607284036
477,Mount Isa,-20.73,139.50,80.60,26,0,8.05,AU,1607283810
502,Goundam,16.41,-3.67,80.60,23,0,8.32,ML,1607284042
504,Suleja,9.18,7.18,86.00,55,0,3.36,NG,1607284042


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
hotel_df = ideal_weather_df
hotel_name = []
hotel_df['Hotel Name'] = hotel_name
hotel_df

ValueError: Length of values does not match length of index

In [75]:
hotel_name = []
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "radius": 5000,
            "types" : "lodging",
            "key": g_key
            "location": f"{lat}, {lng}"
        }
    
        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"


        response = requests.get(base_url, params=params).json()
        
        hotel_df.loc["Hotel Name"] = response["results"][0]["name"]
        print(hotel_name)
             
    except:
        print("No hotel found")
        pass

SyntaxError: invalid syntax (<ipython-input-75-1faa8ab3c89b>, line 9)

In [86]:

target_type = "lodging"
radius = 5000
spots = list(zip(hotel_df.Lat,hotel_df.Lng))
locations = spots - spots[-1]
for location in locations:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#print(location)

    params = {
        "location":location,
        "type": target_type,
        "radius":radius,
        "key":g_key
    
    }
    response =  requests.get(base_url, params = params).json()
    print(json.dumps(response, indent = 4, sort_keys = False))

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [92]:
print(locations)
gmaps.configure(api_key=g_key)
coordinates = locations
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

[(-35.43, -71.67), (-31.63, -71.17), (14.11, -15.55), (-12.37, -40.67), (-20.73, 139.5), (16.41, -3.67), (9.18, 7.18), ('Capy Motel', 'Capy Motel')]


TraitError: Element of the 'location' trait of a Marker instance must be a valid latitude (-90 <= latitude <= 90), but a value of 'Capy Motel' <class 'str'> was specified.